# Lesson 7 ニューラルネットでゲームを攻略するAIをつくろう

## 目次

- Section 1 解説
    - 1.1 強化学習とは
    - 1.2 強化学習の定式化
    - 1.3 Temporal Difference (TD) 学習
    - 1.4 Q学習
    - 1.5 Deep Q学習
    - 1.6 Checkクイズ
- Section 2 実装①
    - 2.1 OpenAI Gymについて
    - 2.2 Colab上でのレンダリングについて
    - 2.3 DQNでCartPole
- Section 3 テクニック・発展的内容
    - 3.1 Prioritized Experience Replay
    - 3.2 Double DQN
    - 3.3 Dueling Network
    - 3.4 Checkクイズ
- Section 4 実装②
    - 4.1 DQN + テクニックでBreakout
- Section 5 ケーススタディ

## Section 3 テクニック・発展的内容

このSectionでは、 DQNにおける精度をさらに向上させるためのテクニックについて説明していきます。

### 3.1 Prioritized Experience Replay

前述のExperience Replayではすべての経験に対して一様にサンプリングしていましたが、 Prioritized Experience ReplayではTD誤差の大きい、 つまり学習する余地の大きい経験に対して重点的にサンプリングします。
こうすることで学習を高速に進めることができます。

具体的には、 経験$i$における誤差

$$
    \delta^{i} = R_{t+1} + \gamma\max_{a'}Q(s',a')-Q(s',a')
$$

を基に、 Repley memory上の各経験の抽出確率を

$$
    P(i) = \frac{{(|\delta^i| + \epsilon)}^{\alpha}}{\sum_{i'}(|\delta^i| + \epsilon)^{\alpha}}
$$

として計算します。 $\alpha$はハイパーパラメータで、 たとえば$\alpha=0.5$などが用いられます。

### 3.2 Double DQN

通常のQ学習では、 教師信号$R_{t+1} + \gamma \max_{a'}Q(s',a')$の計算にmax操作が含まれます。  
しかしながらここで**Maximization Bias**とよばれる問題が発生してしまいます。

これを理解するために簡単な例を用いて説明します。

#### Maximization Bias

下の図のように、 地点Aを始まりとし、 両端を終わりとするような過程を考えてみます。

<img src="./figures/maxm_bias_1.png" width="500mm">

出典: Richard S. Sutton and Andrew G. Barto. 2017. Reinforcement Learning: An Introduction. Second edition. The MIT Press

ここでエージェントはAにおいて行動leftもしくはrightを選ぶことができるとします。

leftを選んでBに到達した場合、 その次のステップでエージェントは多くの選択肢から行動を選びます。
それらの行動の報酬は平均-0.1、 分散1の正規分布に従っているとします。

rightを選んだ場合は報酬0を受け取りエピソードが終わるものとします。

ここで状態Aでのそれぞれの行動に対する真の行動価値関数を考えてみると、
$$
    q(A, \text{left}) = -0.1, \hspace{10mm} q(A, \text{right}) = 0
$$

となり、 これが本来求めたいものです。

しかしQ学習の下では状態Bにおいてmaxを与える行動を選択しようとするので、 推定値は0よりも大きな値となってしまい、 真の推定値との乖離が発生してしまいます。

このmax操作により価値関数を高めに推定してしまう問題を **Maximization Bias** と呼びます。

次のグラフの赤いバーのように、 この乖離は行動の数が増えるにつれて大きくなることが知られています。

<img src="./figures/double_dqn_fig_1.png" width="700mm">

出典: H. v. Hasselt et al. Deep Reinforcement Learning with Double Q-Learning. AAAI. 2016

Double DQNではargmaxをとるネットワークとQ値を計算するネットワークを分けることにより、 この問題を軽減することを試みています(上グラフの青いバー参照)

すなわち教師信号を次のように計算します。

$$
    y^{(i)} = R_{t+1} + \gamma Q(S_{t+1}, \mathrm{arg}\max_{a'}Q(S_{t+1},a';\theta_t);\theta_{\bar{t}})
$$

ここで$\theta_t$はQネットワークのパラメータ、 $\theta_{\bar{t}}$はargmaxをとるネットワークのパラメータです。

実装においては、 argmaxをとるネットワークにはtarget networkを使用します。

### 3.3 Dueling Network

一般的なQ-Networkでは直接Q値を求めていました。
このとき、 各行動間のQ値の差がQ値の絶対値に比べて小さい時、 小さなノイズで大小関係が入れ替わってしまい行動の選択が適切に行われないという状況が考えられます。

**Dueling Network**では、 行動価値関数(Q値)から状態価値関数(V値)の値を引くことにより、 行動間のQ値の差を絶対的なものから相対的なものに変換し、 この問題を軽減することを試みます。

具体的に、 Dueling NetworkではQ値を次のように分解します。
$$
    Q^{\pi}(s,a) = V^{\pi}(s) + A^{\pi}(s,a)
$$

そして最終的なQ値を以下の式で求めます。
$$
    Q(s,a,;\theta,\alpha,\beta) = V(s;\theta,\beta) + \left(A(s,a;\theta,\alpha)-\frac{1}{|\mathcal{A}|}\sum_{a} A(s,a';\theta,\alpha) \right)
$$

下図のように、 最後の全結合層の部分を分離することでこの計算を行います。

<img src="./figures/dueling_network.png" width="500mm">

出典: Z. Want et al. Dueling Network Architectures for Deep Reinforcement Learning. ICML 2016

### 3.4 Checkクイズ

問1. **Prioritized Experience Replay** に関する説明として正しいものを選択肢から一つ選びなさい。

① Replay Memoryから新しい経験を優先的にサンプリングし学習を効率化する手法。  
② Replay Memoryから学習に使った回数が少ない経験を優先的にサンプリングする手法。  
③ Replay MemoryからTD誤差の大きい経験を優先的にサンプリングし学習を効率化する手法。  
④ Replay Memoryから古い経験を優先的にサンプリングし学習を効率化する手法。

問2. Q学習における**Maximization Bias** に関する説明として正しいものを選択肢から一つ選びなさい。

① Q値の推定値の計算でmax操作をおこなうために、Q値を真の値よりも高く推定してしまう問題。  
② Q値の推定値の計算でmax操作をおこなうために、最もQ値の高い行動ばかり選択してしまう問題。  
③ Q値の推定値の計算でmax操作をおこなうために、計算グラフが連続でなくなり勾配が求められなくなってしまう問題。  
④ Q値の推定値の計算でmax操作をおこなうために、行動数が多いときに計算コストが高くなってしまう問題。

### 参考資料

参考となる資料をいくつか挙げます。(★: おすすめ)

1. ★ R. S. Sutton, "Reinforcement Learning: An Introduction", 2018, MIT Press \[[オンライン版(無料)](http://incompleteideas.net/book/the-book-2nd.html)\], \[[amazon](https://www.amazon.com/Reinforcement-Learning-Introduction-Adaptive-Computation/dp/0262193981)\]
    - 強化学習の代表的な入門書です。深層学習についての記述は少ないですが、強化学習の基本的な考え方から、Q学習, Policy Gradient, Planningなど強化学習全般についてわかりやすく記述されています。理論的な厳密さよりも直感的な理解を優先して書かれている印象です。深層強化学習については他の文献で補う必要がありますが、根本となる考え方は重要かつとてもよみやすいので、英語が苦でなければぜひ読むことをおすすめします。
2. C. Szepesvári著, 小山田 他 訳, "速習 強化学習", 2017, 共立出版 \[[amazon](https://www.amazon.co.jp/%E9%80%9F%E7%BF%92-%E5%BC%B7%E5%8C%96%E5%AD%A6%E7%BF%92-%E2%80%95%E5%9F%BA%E7%A4%8E%E7%90%86%E8%AB%96%E3%81%A8%E3%82%A2%E3%83%AB%E3%82%B4%E3%83%AA%E3%82%BA%E3%83%A0%E2%80%95-Csaba-Szepesvari/dp/4320124227)\]
    - 英文書 "Algorithm for Reinforcement Learning" の訳書です。本編と付録に分かれています。本編は理論的な説明に重点が置かれており敷居が高いですが、付録では近年の深層強化学習 (DQN含む) についてまとめてられており, 参考となるでしょう。
3. Deep RL Bootcamp, 2017, Berkeley CA \[[site](https://sites.google.com/view/deep-rl-bootcamp/home?authuser=0)\]
    - 2017夏に行われた深層強化学習のサマースクールです。講義ビデオ・コード・スライドが揃っています。
4. CS294: Deep Reinforcement Learning, UC Berkeley, 2018 \[[site](http://rail.eecs.berkeley.edu/deeprlcourse/)\]
    - UCバークレーでの2018秋の深層強化学習の講座です。